In [1]:
import pandas as pd
import numba as nb
import numpy as np
import re
import ast
import geopandas as gpd

In [ ]:
#VEEEMOSSSS OTRO INTENTO PIPIPI
# Asegurar tipos
afc_25_completo['SECUENCIA'] = afc_25_completo['SECUENCIA'].astype(float)
afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)
afc_25_completo['NombreLinea'] = afc_25_completo['NombreLinea'].astype(str)

afc_25_2_2['SECUENCIA'] = afc_25_2_2['SECUENCIA'].astype(float)
afc_25_2_2['CodigoTarjeta'] = afc_25_2_2['CodigoTarjeta'].astype(str)
afc_25_2_2['NombreLinea'] = afc_25_2_2['NombreLinea'].astype(str)

# Preagrupar la base completa por línea
afc_por_linea = dict(tuple(afc_25_completo.groupby('NombreLinea')))

# Lista para resultados
resultados = []

# Agrupar por tarjeta y línea en la base filtrada
for (tarjeta, linea), grupo_tarjeta_linea in afc_25_2_2.groupby(['CodigoTarjeta', 'NombreLinea']):
    sec_min = grupo_tarjeta_linea['SECUENCIA'].min()
    sec_max = grupo_tarjeta_linea['SECUENCIA'].max()

    # Si no hay recorrido, saltar
    if sec_min == sec_max:
        continue

    # Obtener todos los tramos de esa línea en la base completa
    grupo_linea_completo = afc_por_linea.get(linea, pd.DataFrame())
    if grupo_linea_completo.empty:
        continue

    # Filtrar solo tramos dentro del rango
    tramos_en_rango = grupo_linea_completo[
        (grupo_linea_completo['SECUENCIA'] >= sec_min) &
        (grupo_linea_completo['SECUENCIA'] <= sec_max)
    ]

    tiempo_total = tramos_en_rango['tiempo_estimado_min'].sum()

    resultados.append({
        'CodigoTarjeta': tarjeta,
        'NombreLinea': linea,
        'SecuenciaInicio': sec_min,
        'SecuenciaFin': sec_max,
        'TiempoTotalEstimado_min': round(tiempo_total, 3)
    })

# Crear el DataFrame final
df_tiempos_tarjeta_linea_3 = pd.DataFrame(resultados)



In [ ]:
# INTENTO 1
# Asegurar tipos
afc_25_completo['SECUENCIA'] = afc_25_completo['SECUENCIA'].astype(float)
afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)
afc_25_completo['CodigoLinea'] = afc_25_completo['CodigoLinea'].astype(str)

# Preagrupar la base completa por línea para acceso rápido
afc_por_linea = dict(tuple(afc_25_completo.groupby('CodigoLinea')))

# Resultado final
resultados = []

# Agrupar por tarjeta
for tarjeta, grupo_tarjeta in afc_25_completo.groupby('CodigoTarjeta'):
    for linea in grupo_tarjeta['CodigoLinea'].unique():
        grupo_linea_tarjeta = grupo_tarjeta[grupo_tarjeta['CodigoLinea'] == linea]

        if grupo_linea_tarjeta.empty:
            continue

        sec_min = grupo_linea_tarjeta['SECUENCIA'].min()
        sec_max = grupo_linea_tarjeta['SECUENCIA'].max()

        # Buscar en TODA la línea los tramos entre esas secuencias
        grupo_linea_completo = afc_por_linea.get(linea, pd.DataFrame())

        if grupo_linea_completo.empty:
            continue

        tramos_dentro_de_rango = grupo_linea_completo[
            (grupo_linea_completo['SECUENCIA'] >= sec_min) &
            (grupo_linea_completo['SECUENCIA'] <= sec_max)
        ]

        tiempo_total_estimado = tramos_dentro_de_rango['tiempo_estimado_min'].sum()

        resultados.append({
            'CodigoTarjeta': tarjeta,
            'CodigoLinea': linea,
            'SecuenciaInicio': sec_min,
            'SecuenciaFin': sec_max,
            'TiempoTotalEstimado_min': round(tiempo_total_estimado, 2)
        })

# Convertir a DataFrame
df_tiempos_tarjeta_linea = pd.DataFrame(resultados)


In [ ]:
# INTENTO2
# Asegurar tipos
afc_25_completo['SECUENCIA'] = afc_25_completo['SECUENCIA'].astype(float)
afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)
afc_25_completo['CodigoLinea'] = afc_25_completo['CodigoLinea'].astype(str)

# Paso 1: calcular secuencia mínima y máxima por tarjeta + línea
rangos = (
    afc_25_completo
    .groupby(['CodigoTarjeta', 'CodigoLinea'])['SECUENCIA']
    .agg(SecuenciaInicio='min', SecuenciaFin='max')
    .reset_index()
)

# Paso 2: unir esos rangos con toda la base (para cada validación)
afc_expandidas = afc_25_completo.merge(rangos, on=['CodigoTarjeta', 'CodigoLinea'], how='left')

# Paso 3: filtrar solo tramos dentro del rango
en_rango = afc_expandidas[
    (afc_expandidas['SECUENCIA'] >= afc_expandidas['SecuenciaInicio']) &
    (afc_expandidas['SECUENCIA'] <= afc_expandidas['SecuenciaFin'])
]

# Paso 4: sumar tiempo por tarjeta + línea
df_tiempos_tarjeta_linea_1 = (
    en_rango
    .groupby(['CodigoTarjeta', 'CodigoLinea'], as_index=False)
    .agg({
        'SecuenciaInicio': 'first',
        'SecuenciaFin': 'first',
        'tiempo_estimado_min': 'sum'
    })
    .rename(columns={'tiempo_estimado_min': 'TiempoTotalEstimado_min'})
)

# Redondear si deseas
df_tiempos_tarjeta_linea_1['TiempoTotalEstimado_min'] = df_tiempos_tarjeta_linea_1['TiempoTotalEstimado_min'].round(2)


In [ ]:
#INTENTO3
# Asegurar tipos
afc_25_completo['SECUENCIA'] = afc_25_completo['SECUENCIA'].astype(float)
afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)
afc_25_completo['CodigoLinea'] = afc_25_completo['CodigoLinea'].astype(str)

resultados = []

# Preagrupar TODA la base por línea
afc_por_linea = dict(tuple(afc_25_completo.groupby('NombreLinea')))

# Iterar por cada tarjeta
for tarjeta, grupo_tarjeta in afc_25_completo.groupby('CodigoTarjeta'):
    
    # Ver las líneas donde validó esa tarjeta
    lineas_usadas = grupo_tarjeta['NombreLinea'].unique()

    for linea in lineas_usadas:
        grupo_linea_tarjeta = grupo_tarjeta[grupo_tarjeta['NombreLinea'] == linea]

        if grupo_linea_tarjeta.empty:
            continue

        sec_min = grupo_linea_tarjeta['SECUENCIA'].min()
        sec_max = grupo_linea_tarjeta['SECUENCIA'].max()

        # Buscar en TODA la línea  los tramos dentro del rango
        grupo_linea_completo = afc_por_linea.get(linea, pd.DataFrame())
        if grupo_linea_completo.empty:
            continue

        tramos_en_rango = grupo_linea_completo[
            (grupo_linea_completo['SECUENCIA'] >= sec_min) &
            (grupo_linea_completo['SECUENCIA'] <= sec_max)
        ]

        tiempo_total = tramos_en_rango['tiempo_estimado_min'].sum()

        resultados.append({
            'CodigoTarjeta': tarjeta,
            'NombreLinea': linea,
            'SecuenciaInicio': sec_min,
            'SecuenciaFin': sec_max,
            'TiempoTotalEstimado_min': round(tiempo_total, 3)
        })

df_tiempos_tarjeta_linea_2 = pd.DataFrame(resultados)

In [ ]:
# # Asegurar tipos correctos
# afc_25_completo['SECUENCIA'] = afc_25_completo['SECUENCIA'].astype(float)
# afc_25_completo['CodigoParada'] = afc_25_completo['CodigoParada'].astype(str)
# afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)
# afc_25_completo['CodigoLinea'] = afc_25_completo['CodigoLinea'].astype(str)

# # Lista para resultados
# resultados = []

# # Agrupar por tarjeta y línea
# for (tarjeta, linea), grupo in afc_25_completo.groupby(['CodigoTarjeta', 'CodigoLinea']):
#     sec_min = grupo['SECUENCIA'].min()
#     sec_max = grupo['SECUENCIA'].max()
    
#     # Filtrar todas las filas de esa línea dentro del rango de secuencia (aunque no hayan sido validadas)
#     tramos_de_linea = afc_25_completo[
#         (afc_25_completo['CodigoLinea'] == linea) &
#         (afc_25_completo['SECUENCIA'] >= sec_min) &
#         (afc_25_completo['SECUENCIA'] <= sec_max)
#     ]

#     # Sumar el tiempo estimado dentro del rango
#     tiempo_total_estimado = tramos_de_linea['tiempo_estimado_min'].sum()

#     resultados.append({
#         'CodigoTarjeta': tarjeta,
#         'CodigoLinea': linea,
#         'SecuenciaInicio': sec_min,
#         'SecuenciaFin': sec_max,
#         'TiempoTotalEstimado_min': round(tiempo_total_estimado, 2)
#     })

# # Convertir a DataFrame final
# df_tiempos_por_tarjeta = pd.DataFrame(resultados)


In [ ]:
# Asegura tipos
afc_25_completo['SECUENCIA'] = afc_25_completo['SECUENCIA'].astype(float)
afc_25_completo['CodigoLinea'] = afc_25_completo['CodigoLinea'].astype(str)
afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)

# Agrupamos por tarjeta y línea
def calcular_tiempo(grupo):
    sec_min = grupo['SECUENCIA'].min()
    sec_max = grupo['SECUENCIA'].max()
    
    # Filtrar dentro del mismo grupo (no sobre todo el DataFrame)
    tramos = grupo[(grupo['SECUENCIA'] >= sec_min) & (grupo['SECUENCIA'] <= sec_max)]
    tiempo = tramos['tiempo_estimado_min'].sum()
    
    return pd.Series({
        'SecuenciaInicio': sec_min,
        'SecuenciaFin': sec_max,
        'TiempoTotalEstimado_min': round(tiempo, 2)
    })

# Aplicar por grupo
df_tiempos_por_tarjeta = afc_25_completo.groupby(['CodigoTarjeta', 'CodigoLinea']).apply(calcular_tiempo).reset_index()


In [ ]:
# Asegurar tipos
afc_25_completo = afc_25_completo.copy()

afc_25_completo['SECUENCIA_GLOBAL'] = afc_25_completo['SECUENCIA_GLOBAL'].astype(float)
afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)
afc_25_completo['CodigoLinea'] = afc_25_completo['CodigoLinea'].astype(str)

afc_25_2_2['SECUENCIA_GLOBAL'] = afc_25_2_2['SECUENCIA_GLOBAL'].astype(float)
afc_25_2_2['CodigoTarjeta'] = afc_25_2_2['CodigoTarjeta'].astype(str)
afc_25_2_2['CodigoLinea'] = afc_25_2_2['CodigoLinea'].astype(str)

# Paso 1: agrupar por línea y secuencia para sacar promedio
tiempos_promedio = (
    afc_25_completo
    .groupby(['CodigoLinea', 'SECUENCIA_GLOBAL'], as_index=False)
    .agg(tiempo_medio_min=('tiempo_estimado_min', 'mean'))
)

# Paso 2: guardar resultados
resultados = []

# Paso 3: agrupar por tarjeta y línea desde afc_25_2_2
for (tarjeta, linea), grupo_tarjeta_linea in tqdm(afc_25_2_2.groupby(['CodigoTarjeta', 'CodigoLinea']), desc="Procesando tarjetas"):
    sec_min = grupo_tarjeta_linea['SECUENCIA_GLOBAL'].min()
    sec_max = grupo_tarjeta_linea['SECUENCIA_GLOBAL'].max()

    if sec_min == sec_max:
        continue

    # Filtrar promedios dentro del rango
    tiempos_linea = tiempos_promedio[
        (tiempos_promedio['CodigoLinea'] == linea) &
        (tiempos_promedio['SECUENCIA_GLOBAL'] >= sec_min) &
        (tiempos_promedio['SECUENCIA_GLOBAL'] <= sec_max)
    ]

    tiempo_total = tiempos_linea['tiempo_medio_min'].sum()

    resultados.append({
        'CodigoTarjeta': tarjeta,
        'CodigoLinea': linea,
        'SecuenciaInicio': sec_min,
        'SecuenciaFin': sec_max,
        'TiempoTotalEstimado_min': round(tiempo_total, 3)
    })

# Convertir a DataFrame final
df_tiempos = pd.DataFrame(resultados)

In [ ]:
# OPCIÓN ALTERNATIVA, MÁS PULIDA PERO LENTA 
# # Asegurar tipos
# afc_25_completo = afc_25_completo.copy()
# afc_25_completo['SECUENCIA_GLOBAL'] = afc_25_completo['SECUENCIA_GLOBAL'].astype(float)
# afc_25_completo['CodigoTarjeta'] = afc_25_completo['CodigoTarjeta'].astype(str)
# afc_25_completo['CodigoLinea'] = afc_25_completo['CodigoLinea'].astype(str)

# afc_25_2_2['SECUENCIA_GLOBAL'] = afc_25_2_2['SECUENCIA_GLOBAL'].astype(float)
# afc_25_2_2['CodigoTarjeta'] = afc_25_2_2['CodigoTarjeta'].astype(str)
# afc_25_2_2['CodigoLinea'] = afc_25_2_2['CodigoLinea'].astype(str)

# # Paso: calcular tiempo total secuencial entre sec_min y sec_max
# resultados = []

# for tarjeta, grupo in tqdm(afc_25_2_2.groupby('CodigoTarjeta'), desc="Procesando tarjetas"):
#     grupo = grupo.sort_values('SECUENCIA_GLOBAL')
#     sec_min = grupo['SECUENCIA_GLOBAL'].min()
#     sec_max = grupo['SECUENCIA_GLOBAL'].max()
#     linea = grupo['CodigoLinea'].iloc[0]

#     if sec_min == sec_max:
#         continue

#     tiempo_total = 0
#     sec_actual = int(sec_min)

#     while sec_actual < sec_max:
#         candidatos = afc_25_completo[
#             (afc_25_completo['CodigoLinea'] == linea) &
#             (afc_25_completo['SECUENCIA_GLOBAL'] == sec_actual)
#         ]

#         if candidatos.empty:
#             sec_actual += 1
#             continue

#         if len(candidatos) > 1:
#             tiempos_unicos = candidatos['tiempo_estimado_min'].unique()
#             if len(tiempos_unicos) == 1:
#                 tiempo_total += tiempos_unicos[0]
#             else:
#                 print(f"[Ambiguo] Tarjeta {tarjeta}, línea {linea}, secuencia {sec_actual}: múltiples tiempos distintos encontrados: {tiempos_unicos}")
#             sec_actual += 1
#             continue

#         # Si hay solo una fila
#         tiempo_total += candidatos.iloc[0]['tiempo_estimado_min']
#         sec_actual += 1

#     resultados.append({
#         'CodigoTarjeta': tarjeta,
#         'CodigoLinea': linea,
#         'SecuenciaInicio': sec_min,
#         'SecuenciaFin': sec_max,
#         'TiempoTotalEstimado_min': round(tiempo_total, 3)
#     })

# df_tiempos_tarjeta_linea_corregido = pd.DataFrame(resultados)